In [1]:
import numpy as np
import librosa
from sklearn.mixture import GaussianMixture
import os

In [2]:
# Parse the metadata file (use only the first 5000 samples for training)
def parse_metadata(file_path, train_limit=5000):
    overlapped_files = []
    non_overlapped_files = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Skip the header
        lines = lines[:train_limit]  # Use only the first `train_limit` lines for training
        
        for line in lines:
            parts = line.strip().split(",")
            audio_file = parts[0].strip()
            tag = parts[3].strip()
            
            if tag == "overlapped":
                overlapped_files.append(audio_file)
            elif tag == "non-overlapped":
                non_overlapped_files.append(audio_file)
    
    return overlapped_files, non_overlapped_files

# Main processing (no changes needed here)

# Extract MFCC features
def extract_features(audio_files, audio_dir):
    features = []
    for audio_file in audio_files:
        audio_path = os.path.join(audio_dir, audio_file)
        try:
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc_mean = np.mean(mfcc, axis=1)  # Take the mean across time
            features.append(mfcc_mean)
        except Exception as e:
            print(f"Error processing {audio_file}: {e}")
    
    return np.array(features)

# Train GMM
def train_gmm(features, n_components=64):
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(features)
    return gmm
def main(metadata_file, audio_dir):
    overlapped_files, non_overlapped_files = parse_metadata(metadata_file)
    
    print("Extracting features for overlapped samples...")
    overlapped_features = extract_features(overlapped_files, audio_dir)
    
    print("Extracting features for non-overlapped samples...")
    non_overlapped_features = extract_features(non_overlapped_files, audio_dir)
    
    print("Training GMM for overlapped samples...")
    gmm_overlapped = train_gmm(overlapped_features)
    
    print("Training GMM for non-overlapped samples...")
    gmm_non_overlapped = train_gmm(non_overlapped_features)
    
    return gmm_overlapped, gmm_non_overlapped

# Set paths and run
metadata_file = "output.txt"
audio_dir = "output_audio"  # Replace with the directory containing audio files

gmm_overlapped, gmm_non_overlapped = main(metadata_file, audio_dir)

# Output GMM parameters
print("Lambda(o) - Overlapped GMM Parameters:")
print(f"Weights: {gmm_overlapped.weights_}")
print(f"Means: {gmm_overlapped.means_}")
print(f"Covariances: {gmm_overlapped.covariances_}")

print("\nLambda(n) - Non-Overlapped GMM Parameters:")
print(f"Weights: {gmm_non_overlapped.weights_}")
print(f"Means: {gmm_non_overlapped.means_}")
print(f"Covariances: {gmm_non_overlapped.covariances_}")


Extracting features for overlapped samples...
Extracting features for non-overlapped samples...
Training GMM for overlapped samples...
Training GMM for non-overlapped samples...
Lambda(o) - Overlapped GMM Parameters:
Weights: [0.01329259 0.0209339  0.01630009 0.00867352 0.01014924 0.03377972
 0.01174427 0.01730057 0.01568165 0.01349352 0.01577387 0.01105084
 0.01951876 0.01753722 0.01874325 0.01570928 0.00908095 0.01704024
 0.02870652 0.01276261 0.01754971 0.01862003 0.014863   0.01146744
 0.0242558  0.02067348 0.01611894 0.00870844 0.01921076 0.00394945
 0.01411236 0.01145418 0.02251828 0.01649217 0.01497737 0.0062914
 0.01360912 0.01526147 0.02573341 0.01438231 0.02708528 0.01518644
 0.00710769 0.01359705 0.013887   0.01862804 0.0169028  0.01854509
 0.01211992 0.01782073 0.01495779 0.012389   0.01711945 0.01341637
 0.01308313 0.00236967 0.01924484 0.02371773 0.01586241 0.02903507
 0.01644716 0.00813147 0.01068997 0.00513417]
Means: [[-3.03421880e+02  1.39577532e+02 -9.26846358e+00  3

In [3]:
# Function to save GMM parameters to a text file
def save_gmm_parameters(gmm_overlapped, gmm_non_overlapped, output_file):
    with open(output_file, "w") as f:
        # Save parameters for overlapped samples
        f.write("Lambda(o) - Overlapped GMM Parameters:\n")
        f.write(f"Weights:\n{gmm_overlapped.weights_}\n")
        f.write(f"Means:\n{gmm_overlapped.means_}\n")
        f.write(f"Covariances:\n{gmm_overlapped.covariances_}\n\n")
        
        # Save parameters for non-overlapped samples
        f.write("Lambda(n) - Non-Overlapped GMM Parameters:\n")
        f.write(f"Weights:\n{gmm_non_overlapped.weights_}\n")
        f.write(f"Means:\n{gmm_non_overlapped.means_}\n")
        f.write(f"Covariances:\n{gmm_non_overlapped.covariances_}\n")
    print(f"GMM parameters saved to {output_file}")




# Save the GMM parameters to a text file
output_file = "gmm_parameters.txt"
save_gmm_parameters(gmm_overlapped, gmm_non_overlapped, output_file)


GMM parameters saved to gmm_parameters.txt


In [4]:
import joblib

# Assuming you have trained these models
# gmm_overlapped = GaussianMixture(n_components=64).fit(X_overlapped)
# gmm_non_overlapped = GaussianMixture(n_components=64).fit(X_non_overlapped)

# Save both GMM models together
joblib.dump((gmm_overlapped, gmm_non_overlapped), "gmm_models.pkl")

print("GMM models saved successfully!")


GMM models saved successfully!


In [6]:
import os
import numpy as np
import librosa
from sklearn.metrics import confusion_matrix

# Evaluate probabilities and calculate accuracy, confusion matrix, miss rate, and false alarm rate
def evaluate_gmm(gmm_overlapped, gmm_non_overlapped, eval_files, eval_labels, audio_dir):
    correct_predictions = 0
    total_samples = len(eval_files)
    
    predictions = []
    ground_truths = []
    
    results = []
    
    for i, audio_file in enumerate(eval_files):
        audio_path = os.path.join(audio_dir, audio_file)
        try:
            # Extract features
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc_mean = np.mean(mfcc, axis=1).reshape(1, -1)  # Reshape for GMM
            
            # Calculate log-likelihoods
            log_likelihood_overlapped = gmm_overlapped.score(mfcc_mean)
            log_likelihood_non_overlapped = gmm_non_overlapped.score(mfcc_mean)
            
            # Predict class based on higher likelihood
            predicted_label = "overlapped" if log_likelihood_overlapped > log_likelihood_non_overlapped else "non-overlapped"
            ground_truth = eval_labels[i]
            score = abs(log_likelihood_overlapped - log_likelihood_non_overlapped)
            
            # Check correctness
            result = predicted_label == ground_truth
            if result:
                correct_predictions += 1
            
            # Save prediction results
            results.append(f"{audio_file}\t{predicted_label}\t{ground_truth}\t{score:.3f}\t{result}\n")
            
            # Store for confusion matrix calculation
            predictions.append(predicted_label)
            ground_truths.append(ground_truth)
        
        except Exception as e:
            print(f"Error processing {audio_file}: {e}")

    # Calculate accuracy
    accuracy = (correct_predictions / total_samples) * 100
    
    # Save results to file
    with open("output_results.txt", "w") as f:
        f.write("filename\tpredicted\tgroundtruth\tscore\tresult\n")
        f.writelines(results)

    # Compute confusion matrix
    labels = ["overlapped", "non-overlapped"]
    cm = confusion_matrix(ground_truths, predictions, labels=labels)
    
    # Extract values from confusion matrix
    TP = cm[0, 0]  # True Positive (Predicted overlapped, actual overlapped)
    FN = cm[0, 1]  # False Negative (Predicted non-overlapped, actual overlapped) - Miss rate
    FP = cm[1, 0]  # False Positive (Predicted overlapped, actual non-overlapped) - False alarm rate
    TN = cm[1, 1]  # True Negative (Predicted non-overlapped, actual non-overlapped)

    # Compute Miss Rate & False Alarm Rate
    miss_rate = FN / (TP + FN) if (TP + FN) > 0 else 0
    false_alarm_rate = FP / (FP + TN) if (FP + TN) > 0 else 0

    # Save confusion matrix and error rates
    with open("confusion_matrix.txt", "w") as f:
        f.write("Confusion Matrix:\n")
        f.write(f"                Predicted Overlapped  Predicted Non-Overlapped\n")
        f.write(f"Actual Overlapped      {TP}                   {FN}\n")
        f.write(f"Actual Non-Overlapped  {FP}                   {TN}\n\n")
        f.write(f"Miss Rate: {miss_rate:.4f}\n")
        f.write(f"False Alarm Rate: {false_alarm_rate:.4f}\n")
        f.write(f"Accuracy: {accuracy:.2f}%\n")

    return accuracy

# Parse metadata for evaluation
def parse_metadata_for_eval(file_path, start_line=5001):
    eval_files = []
    eval_labels = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Skip the header
        lines = lines[start_line - 1:]  # Take lines starting from 5001
        
        for line in lines:
            parts = line.strip().split(",")
            audio_file = parts[0].strip()
            label = parts[3].strip()
            
            eval_files.append(audio_file)
            eval_labels.append(label)
    
    return eval_files, eval_labels

# Main processing for evaluation
metadata_file = "output.txt"
audio_dir = "output_audio"  # Replace with the directory containing audio files

# Get evaluation data
eval_files, eval_labels = parse_metadata_for_eval(metadata_file)

# Evaluate and calculate accuracy
accuracy = evaluate_gmm(gmm_overlapped, gmm_non_overlapped, eval_files, eval_labels, audio_dir)

# Print accuracy
print(f"Evaluation Accuracy: {accuracy:.2f}%")
print("Results saved in 'output_results.txt' and 'confusion_matrix.txt'")


Evaluation Accuracy: 93.53%
